<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/similar_search_models_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install tensorflow==2.8.0 --quiet
!pip install transformers==4.15.0 --quiet
!pip install scikit-learn --quiet
!pip install faiss-gpu transformers torch --quiet

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from transformers import BertTokenizer, TFBertModel
from transformers import logging
logging.set_verbosity_error()
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
import re

In [3]:
from google.colab import drive

# mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
from tensorflow.keras.models import load_model

# model_v3
# model_v4
# model_freeze_classifer

custom_objects = {'TFBertModel': TFBertModel}
model_v3data = load_model('./drive/MyDrive/Two_tower_level4_PII/two_tower_bert_v3.h5', custom_objects=custom_objects)
model_v4data = load_model('./drive/MyDrive/Two_tower_level4_PII/two_tower_bert_v4.h5', custom_objects=custom_objects)
model_v5data = load_model('./drive/MyDrive/Two_tower_level4_PII/two_tower_bert_v5.h5', custom_objects=custom_objects)
model_freeze_classifer_v3v5 = load_model('./drive/MyDrive/Two_tower_level4_PII/two_tower_bert_freezeclassifer_v3v5.h5', custom_objects=custom_objects)
model_freeze_classifer_v3v4 = load_model('./drive/MyDrive/Two_tower_level4_PII/two_tower_bert_freezeclassifer_v3v4.h5', custom_objects=custom_objects)
model_freeze_classifer_v4v5 = load_model('./drive/MyDrive/Two_tower_level4_PII/two_tower_bert_freezeclassifer_v4v5.h5', custom_objects=custom_objects)

In [10]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
model_v5data.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]',     

In [8]:
# cls extraction
def generate_embedding_cls(text, model):
  bert_embedding_model = Model(inputs=[model.input],
                               outputs=model.get_layer('tf.__operators__.getitem').output)
  bert_train_tokenized = bert_tokenizer(text,
                                        max_length=128,
                                        truncation=True,
                                        padding='max_length',
                                        return_tensors='tf')
  bert_train_inputs = [bert_train_tokenized.input_ids,
                       bert_train_tokenized.token_type_ids,
                       bert_train_tokenized.attention_mask]
  return bert_embedding_model.predict(bert_train_inputs)


In [7]:
# maximum pooling

def generate_embedding_max(text, model, bert_name):
    # same input
    input_ids = model.get_layer('input_ids').input
    attention_mask = model.get_layer('attention_mask').input
    token_type_ids = model.get_layer('token_type_ids').input
    #get bert output
    bert_layer = model.get_layer(bert_name)
    bert_output = bert_layer([input_ids, attention_mask, token_type_ids])
    # Apply max pooling on the sequence output (last_hidden_state: (batch, sequence, embedding dimensions))
    pooled_output = GlobalMaxPooling1D()(bert_output[0])
    # Create a new model for embedding extraction
    embedding_model = Model(inputs=[input_ids, attention_mask, token_type_ids],
                            outputs=pooled_output)

    bert_train_tokenized = bert_tokenizer(text,
                                        max_length=128,
                                        truncation=True,
                                        padding='max_length',
                                        return_tensors='tf')
    bert_train_inputs = [bert_train_tokenized.input_ids,
                       bert_train_tokenized.token_type_ids,
                       bert_train_tokenized.attention_mask]

    return embedding_model.predict(bert_train_inputs)

In [ ]:
#check the size of embeddings
text = """As Jose, a student at Johnnyfurt Higher Institute, I have learned the importance of time management and organization through my academic journey. With my student ID number DNM7052, I have had the opportunity to engage in various lI am doing H1B SPONSOR FOR L1/L2/OPT at US, NY, New York  """
embeddings_v3 = generate_embedding_cls(text, model_v3data)
embeddings_v4 = generate_embedding_cls(text, model_v4data)
embeddings_v5 = generate_embedding_cls(text, model_v5data)

embeddings_freeze_classifer_v3v4 = generate_embedding_cls(text, model_freeze_classifer_v3v4)
embeddings_freeze_classifer_v4v5 = generate_embedding_cls(text, model_freeze_classifer_v4v5)
embeddings_freeze_classifer_v3v5 = generate_embedding_cls(text, model_freeze_classifer_v3v5)

embeddings_v5_max = generate_embedding_max(text, model_v5data, bert_name='tf_bert_model_2')
embeddings_freeze_classifer_v4v5_max = generate_embedding_max(text, model_freeze_classifer_v4v5, bert_name='tf_bert_model_5')

print(embeddings_v3.shape, embeddings_v4.shape, embeddings_v5.shape, embeddings_freeze_classifer_v3v4.shape, embeddings_freeze_classifer_v4v5.shape, embeddings_freeze_classifer_v3v5.shape, embeddings_v5_max.shape, embeddings_freeze_classifer_v4v5_max.shape)

In [11]:
# (768,1) to 768
def embedding_vectordatabase(embedding):
  embeddings = np.array(embedding, dtype="float32")
  embeddings = embeddings.reshape(-1, embeddings.shape[-1])
  return embeddings

In [22]:
# create vector databases
import faiss
import numpy as np

dimension = 768  # Dimension of BERT-base embeddings

index_cls_v3 = faiss.IndexFlatL2(dimension)
index_cls_v4 = faiss.IndexFlatL2(dimension)
index_cls_v5 = faiss.IndexFlatL2(dimension)

index_cls_freeze_classifier_v3v4 = faiss.IndexFlatL2(dimension)
index_cls_freeze_classifier_v3v5 = faiss.IndexFlatL2(dimension)
index_cls_freeze_classifier_v4v5 = faiss.IndexFlatL2(dimension)

index_v5_max = faiss.IndexFlatL2(dimension)
index_freeze_classifer_v4v5_max = faiss.IndexFlatL2(dimension)

In [23]:
# embedding: model_v3 + cls
# embedding: model_v4 + cls
# embedding: model_freeze_classifer + cls
# embedding: model_freeze_classifer + max pooling


level_four_job = ["Competitive salary for iPhone project management roles",
                  "Relocation benefits for Apple Cupertino campus positions",
                  "Exclusive health and wellness benefits for iPhone development team members",
                  "Stock options and bonuses for iPhone project milestones",
                  "Collaborative workspaces at Apple's Silicon Valley locations",
                  "Professional development programs for iOS project leads",
                  "Confidentiality agreements for upcoming iPhone projects",
                  "Global travel opportunities for iPhone supplier audits",
                  "Flexible working hours for Apple's product development teams",
                  "Team-building retreats for iPhone project managers"]

for input in level_four_job:
  embeddings = generate_embedding_cls(input, model_v3data)
  index_cls_v3.add(embedding_vectordatabase(embeddings))

  embeddings = generate_embedding_cls(input, model_v4data)
  index_cls_v4.add(embedding_vectordatabase(embeddings))

  embeddings = generate_embedding_cls(input, model_v5data)
  index_cls_v5.add(embedding_vectordatabase(embeddings))

  embeddings = generate_embedding_cls(input, model_freeze_classifer_v3v4)
  index_cls_freeze_classifier_v3v4.add(embedding_vectordatabase(embeddings))

  embeddings = generate_embedding_cls(input, model_freeze_classifer_v3v5)
  index_cls_freeze_classifier_v3v5.add(embedding_vectordatabase(embeddings))

  embeddings = generate_embedding_cls(input, model_freeze_classifer_v4v5)
  index_cls_freeze_classifier_v4v5.add(embedding_vectordatabase(embeddings))

  embeddings = generate_embedding_max(text, model_v5data, bert_name='tf_bert_model_2')
  index_v5_max.add(embedding_vectordatabase(embeddings))

  embeddings = generate_embedding_max(text, model_freeze_classifer_v4v5, bert_name='tf_bert_model_5')
  index_freeze_classifer_v4v5_max.add(embedding_vectordatabase(embeddings))

print(index_cls_v3.ntotal, index_cls_v4.ntotal, index_cls_v5.ntotal, index_cls_freeze_classifier_v3v4.ntotal, index_cls_freeze_classifier_v3v5.ntotal, index_cls_freeze_classifier_v4v5.ntotal, index_v5_max.ntotal, index_freeze_classifer_v4v5_max.ntotal)

10 10 10 10 10 10 10 10


In [28]:
# compare distance for a input and four vector database.
# based on "Relocation benefits for Apple Cupertino campus positions", ask GPT to generate a prompt (#1)
text = """As Taylor Morgan, I am preparing to discuss my health concerns and lifestyle changes due to my recent relocation for a position at Apple's Cupertino campus.
"""
# Indices of closest embeddings for v3 model: [[1 6 7]]
# Distances for v3 model: [[462.62048 557.87604 562.0688 ]]
# Indices of closest embeddings for v4 model: [[8 7 3]]
# Distances for v4 model: [[834.939   842.25964 842.5376 ]]
# Indices of closest embeddings for v5 model: [[1 6 4]]
# Distances for v5 model: [[149.1654  152.35236 153.35999]]
# Indices of closest embeddings for fc_v3v4 model: [[0 3 8]]
# Distances for fc_v3v4 model: [[664.42206 664.51105 665.29034]]
# Indices of closest embeddings for fc_v4v5 model: [[4 6 2]]
# Distances for fc_v4v5 model: [[521.54614 548.7949  550.61694]]
# Indices of closest embeddings for fc_v3v5 model: [[4 6 1]]
# Distances for fc_v3v5 model: [[387.16553 421.3154  462.21545]]
# Indices of closest embeddings for fcm_v5 model: [[1 2 0]]
# Distances for fcm_v5 model: [[94.93431 94.93431 94.93431]]
# Indices of closest embeddings for fcm_v4v5 model: [[1 2 0]]
# Distances for fcm_v4v5 model: [[99.05984 99.05984 99.05984]]

#based on "Global travel opportunities for iPhone supplier audits" (#7)
text = """
As Alex Rivera, I am reporting on the health implications and challenges faced due to my role involving global travel for iPhone supplier audits.
"""
# Indices of closest embeddings for v3 model: [[1 7 6]]
# Distances for v3 model: [[175.7427  207.36827 233.06667]]
# Indices of closest embeddings for v4 model: [[7 8 2]]
# Distances for v4 model: [[621.7367  626.84393 627.2821 ]]
# Indices of closest embeddings for v5 model: [[6 7 2]]
# Distances for v5 model: [[45.748867 49.99069  51.427288]]
# Indices of closest embeddings for fc_v3v4 model: [[4 2 8]]
# Distances for fc_v3v4 model: [[1.6468006 1.7377512 1.7633522]]
# Indices of closest embeddings for fc_v4v5 model: [[4 2 6]]
# Distances for fc_v4v5 model: [[59.87197 69.83478 71.50404]]
# Indices of closest embeddings for fc_v3v5 model: [[4 6 1]]
# Distances for fc_v3v5 model: [[ 67.106606 105.07139  140.9225  ]]
# Indices of closest embeddings for fcm_v5 model: [[1 2 0]]
# Distances for fcm_v5 model: [[0. 0. 0.]]
# Indices of closest embeddings for fcm_v4v5 model: [[1 2 0]]
# Distances for fcm_v4v5 model: [[0. 0. 0.]]

# 8. longer? "Flexible working hours for Apple's product development teams"
text = """
As Jordan Kim, I'm addressing the impact of flexible working hours on my health as part of Apple's product development team. Since the implementation of flexible working hours, I've noticed significant changes in my daily routine, some beneficial and others challenging. The ability to adjust my work hours has afforded me a greater work-life balance, allowing for more time spent with family and engaging in personal hobbies. However, without the structure of fixed working hours, I've found it difficult to establish a consistent sleep schedule, leading to occasional insomnia and varying energy levels throughout the day.
"""
# Indices of closest embeddings for v3 model: [[1 6 7]]
# Distances for v3 model: [[442.27026 531.46814 534.8041 ]]
# Indices of closest embeddings for v4 model: [[8 6 3]]
# Distances for v4 model: [[849.8129  857.4177  858.35767]]
# Indices of closest embeddings for v5 model: [[2 6 9]]
# Distances for v5 model: [[ 98.07463  99.06271 100.05426]]
# Indices of closest embeddings for fc_v3v4 model: [[1 7 8]]
# Distances for fc_v3v4 model: [[156.9984  157.8388  158.15071]]
# Indices of closest embeddings for fc_v4v5 model: [[4 2 6]]
# Distances for fc_v4v5 model: [[566.60895 589.8434  590.8391 ]]
# Indices of closest embeddings for fc_v3v5 model: [[4 6 1]]
# Distances for fc_v3v5 model: [[215.80788 261.38766 304.94348]]
# Indices of closest embeddings for fcm_v5 model: [[1 2 0]]
# Distances for fcm_v5 model: [[214.11975 214.11975 214.11975]]
# Indices of closest embeddings for fcm_v4v5 model: [[1 2 0]]
# Distances for fcm_v4v5 model: [[180.59593 180.59593 180.59593]]

# 8. sentence level
text = """
As Jordan Kim, I'm addressing the impact of flexible working hours on my health as part of Apple's product development team.
"""
# Indices of closest embeddings for v3 model: [[7 4 1]]
# Distances for v3 model: [[141.25726 142.04306 148.1364 ]]
# Indices of closest embeddings for v4 model: [[5 9 8]]
# Distances for v4 model: [[434.44342 434.8328  437.09268]]
# Indices of closest embeddings for v5 model: [[8 6 4]]
# Distances for v5 model: [[27.352757 27.495752 29.006763]]
# Indices of closest embeddings for fc_v3v4 model: [[4 8 2]]
# Distances for fc_v3v4 model: [[2.7517064 2.8037987 2.904843 ]]
# Indices of closest embeddings for fc_v4v5 model: [[4 2 6]]
# Distances for fc_v4v5 model: [[79.770325 93.841736 99.04136 ]]
# Indices of closest embeddings for fc_v3v5 model: [[4 6 1]]
# Distances for fc_v3v5 model: [[ 63.460854 107.30418  140.4897  ]]
# Indices of closest embeddings for fcm_v5 model: [[1 2 0]]
# Distances for fcm_v5 model: [[32.360844 32.360844 32.360844]]
# Indices of closest embeddings for fcm_v4v5 model: [[1 2 0]]
# Distances for fcm_v4v5 model: [[25.956692 25.956692 25.956692]]


# diagnosis, no job related information
text = "I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."
# Indices of closest embeddings for v3 model: [[1 6 7]]
# Distances for v3 model: [[765.25256 848.5805  867.85834]]
# Indices of closest embeddings for v4 model: [[8 7 3]]
# Distances for v4 model: [[829.60986 832.6284  833.5238 ]]
# Indices of closest embeddings for v5 model: [[3 1 4]]
# Distances for v5 model: [[537.1964  539.3674  540.19867]]
# Indices of closest embeddings for fc_v3v4 model: [[0 3 7]]
# Distances for fc_v3v4 model: [[1051.2994 1052.177  1052.7937]]
# Indices of closest embeddings for fc_v4v5 model: [[4 6 2]]
# Distances for fc_v4v5 model: [[655.0598  680.81445 682.2945 ]]
# Indices of closest embeddings for fc_v3v5 model: [[4 6 1]]
# Distances for fc_v3v5 model: [[635.7446 677.0921 719.471 ]]
# Indices of closest embeddings for fcm_v5 model: [[1 2 0]]
# Distances for fcm_v5 model: [[227.31435 227.31435 227.31435]]
# Indices of closest embeddings for fcm_v4v5 model: [[1 2 0]]
# Distances for fcm_v4v5 model: [[234.64517 234.64517 234.64517]]

embeddings_v3 = generate_embedding_cls(text, model_v3data)
embeddings_v4 = generate_embedding_cls(text, model_v4data)
embeddings_v5 = generate_embedding_cls(text, model_v5data)

embeddings_freeze_classifer_v3v4 = generate_embedding_cls(text, model_freeze_classifer_v3v4)
embeddings_freeze_classifer_v4v5 = generate_embedding_cls(text, model_freeze_classifer_v4v5)
embeddings_freeze_classifer_v3v5 = generate_embedding_cls(text, model_freeze_classifer_v3v5)

embeddings_v5_max = generate_embedding_max(text, model_v5data, bert_name='tf_bert_model_2')
embeddings_freeze_classifer_v4v5_max = generate_embedding_max(text, model_freeze_classifer_v4v5, bert_name='tf_bert_model_5')

# Search the index
k = 3
distances_v3, indices_v3 = index_cls_v3.search(embeddings_v3, k)
distances_v4, indices_v4 = index_cls_v4.search(embeddings_v4, k)
distances_v5, indices_v5 = index_cls_v5.search(embeddings_v5, k)

distances_fc_v3v4, indices_fc_v3v4 = index_cls_freeze_classifier_v3v4.search(embeddings_freeze_classifer_v3v4, k)
distances_fc_v4v5, indices_fc_v4v5 = index_cls_freeze_classifier_v4v5.search(embeddings_freeze_classifer_v4v5, k)
distances_fc_v3v5, indices_fc_v3v5 = index_cls_freeze_classifier_v3v5.search(embeddings_freeze_classifer_v3v5, k)

distances_fcm_v5, indices_fcm_v5 = index_v5_max.search(embeddings_v5_max, k)
distances_fcm_v4v5, indices_fcm_v4v5 = index_freeze_classifer_v4v5_max.search(embeddings_freeze_classifer_v4v5_max, k)

print("Indices of closest embeddings for v3 model:", indices_v3)
print("Distances for v3 model:", distances_v3)

print("Indices of closest embeddings for v4 model:", indices_v4)
print("Distances for v4 model:", distances_v4)

print("Indices of closest embeddings for v5 model:", indices_v5)
print("Distances for v5 model:", distances_v5)

print("Indices of closest embeddings for fc_v3v4 model:", indices_fc_v3v4)
print("Distances for fc_v3v4 model:", distances_fc_v3v4)

print("Indices of closest embeddings for fc_v4v5 model:", indices_fc_v4v5)
print("Distances for fc_v4v5 model:", distances_fc_v4v5)

print("Indices of closest embeddings for fc_v3v5 model:", indices_fc_v3v5)
print("Distances for fc_v3v5 model:", distances_fc_v3v5)

print("Indices of closest embeddings for fcm_v5 model:", indices_fcm_v5)
print("Distances for fcm_v5 model:", distances_fcm_v5)

print("Indices of closest embeddings for fcm_v4v5 model:", indices_fcm_v4v5)
print("Distances for fcm_v4v5 model:", distances_fcm_v4v5)


Indices of closest embeddings for v3 model: [[1 6 7]]
Distances for v3 model: [[765.25256 848.5805  867.85834]]
Indices of closest embeddings for v4 model: [[8 7 3]]
Distances for v4 model: [[829.60986 832.6284  833.5238 ]]
Indices of closest embeddings for v5 model: [[3 1 4]]
Distances for v5 model: [[537.1964  539.3674  540.19867]]
Indices of closest embeddings for fc_v3v4 model: [[0 3 7]]
Distances for fc_v3v4 model: [[1051.2994 1052.177  1052.7937]]
Indices of closest embeddings for fc_v4v5 model: [[4 6 2]]
Distances for fc_v4v5 model: [[655.0598  680.81445 682.2945 ]]
Indices of closest embeddings for fc_v3v5 model: [[4 6 1]]
Distances for fc_v3v5 model: [[635.7446 677.0921 719.471 ]]
Indices of closest embeddings for fcm_v5 model: [[1 2 0]]
Distances for fcm_v5 model: [[227.31435 227.31435 227.31435]]
Indices of closest embeddings for fcm_v4v5 model: [[1 2 0]]
Distances for fcm_v4v5 model: [[234.64517 234.64517 234.64517]]


In [ ]:
# split the prompt
# a clear blacklist, be specific
# setence level seems off, but if better than paragraph.. v
# yes/no OK v
# accurate? doubtful -> try split the prompt and clear blacklist
# It seems v5 is the best so far.

In [5]:
# evaluation of two_tower_bert_v5_max (trained by token after max pooling )
from tensorflow.keras.models import load_model

custom_objects = {'TFBertModel': TFBertModel}
two_tower_bert_v5_max = load_model('./drive/MyDrive/Two_tower_level4_PII/two_tower_bert_v5_max.h5', custom_objects=custom_objects)
two_tower_bert_v5_max.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]',     

In [12]:
text = """As Taylor Morgan, I am preparing to discuss my health concerns and lifestyle changes due to my recent relocation for a position at Apple's Cupertino campus.
"""
embeddings_v5_max = generate_embedding_max(text, two_tower_bert_v5_max, bert_name='tf_bert_model')
embeddings_v5_max.shape

(1, 768)

In [17]:
# create vector databases
import faiss
import numpy as np

dimension = 768
index_embeddings_v5_max = faiss.IndexFlatL2(dimension)
level_four_job = ["Competitive salary for iPhone project management roles",
                  "Relocation benefits for Apple Cupertino campus positions",
                  "Exclusive health and wellness benefits for iPhone development team members",
                  "Stock options and bonuses for iPhone project milestones",
                  "Collaborative workspaces at Apple's Silicon Valley locations",
                  "Professional development programs for iOS project leads",
                  "Confidentiality agreements for upcoming iPhone projects",
                  "Global travel opportunities for iPhone supplier audits",
                  "Flexible working hours for Apple's product development teams",
                  "Team-building retreats for iPhone project managers"]

for input in level_four_job:
  embeddings = generate_embedding_max(text, two_tower_bert_v5_max, bert_name='tf_bert_model')
  index_embeddings_v5_max.add(embedding_vectordatabase(embeddings))
print(index_embeddings_v5_max.ntotal)

10


In [23]:
# based on "Relocation benefits for Apple Cupertino campus positions", ask GPT to generate a prompt (#1)
text = """As Taylor Morgan, I am preparing to discuss my health concerns and lifestyle changes due to my recent relocation for a position at Apple's Cupertino campus.
"""
# Indices of closest embeddings for v5 model: [[1 6 4]]
# Distances for v5 model: [[149.1654  152.35236 153.35999]]
# Indices of closest embeddings for v5 max model: [[1 2 0]]
# Distances for v5 max model: [[0. 0. 0.]]

#based on "Global travel opportunities for iPhone supplier audits" (#7)
text = """
As Alex Rivera, I am reporting on the health implications and challenges faced due to my role involving global travel for iPhone supplier audits.
"""
# Indices of closest embeddings for v5 model: [[6 7 2]]
# Distances for v5 model: [[45.748867 49.99069  51.427288]]
# Indices of closest embeddings for v5 max model: [[1 2 0]]
# Distances for v5 max model: [[121.72088 121.72088 121.72088]]


# 8. longer? "Flexible working hours for Apple's product development teams"
text = """
As Jordan Kim, I'm addressing the impact of flexible working hours on my health as part of Apple's product development team. Since the implementation of flexible working hours, I've noticed significant changes in my daily routine, some beneficial and others challenging. The ability to adjust my work hours has afforded me a greater work-life balance, allowing for more time spent with family and engaging in personal hobbies. However, without the structure of fixed working hours, I've found it difficult to establish a consistent sleep schedule, leading to occasional insomnia and varying energy levels throughout the day.
"""
# Indices of closest embeddings for v5 model: [[2 6 9]]
# Distances for v5 model: [[ 98.07463  99.06271 100.05426]]
# Indices of closest embeddings for v5 max model: [[1 2 0]]
# Distances for v5 max model: [[165.76381 165.76381 165.76381]]


# # 8. sentence level
# text = """
# As Jordan Kim, I'm addressing the impact of flexible working hours on my health as part of Apple's product development team.
# """
# # Indices of closest embeddings for v5 model: [[8 6 4]]
# # Distances for v5 model: [[27.352757 27.495752 29.006763]]



# # diagnosis, no job related information
# text = "I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."
# # Indices of closest embeddings for v5 model: [[3 1 4]]
# # Distances for v5 model: [[537.1964  539.3674  540.19867]]


k=3
embeddings_v5_max = generate_embedding_max(text, two_tower_bert_v5_max, bert_name='tf_bert_model')
distances_v5_max, indices_v5_max = index_embeddings_v5_max.search(embeddings_v5_max, k)

print("Indices of closest embeddings for v5 max model:", indices_v5_max)
print("Distances for v5 max model:", distances_v5_max)

Indices of closest embeddings for v5 max model: [[1 2 0]]
Distances for v5 max model: [[165.76381 165.76381 165.76381]]


1. v5 vs. v5 trained with max_pooling.
3. chunk prompt and do the evaluation.